In [19]:
%load_ext autoreload
%autoreload 
import sys
sys.path.append("../")
import X_py_boost



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Util

In [7]:
import cupy as cp

def to_cat(y):
    
    y_r = cp.array([y[i, :].argmax() for i in range(y.shape[0])])
    return(y_r)

def my_acc(y, pred):
    y_t = cp.array(y[:, cp.newaxis])
    
    return((y.reshape(y.shape[0]) == to_cat(pred).reshape(y.shape[0])).mean())

import plotly.io as pio
import plotly.graph_objects as go
import plotly.express as px
from plotly.offline import init_notebook_mode, iplot
import numpy as np

init_notebook_mode(connected=True)


def get_results(model):
    return list(np.array(model.history).reshape(len(model.history)))

### Model

In [3]:
import tensorflow.keras as keras

(x, y), (a, b) = keras.datasets.mnist.load_data()

x = x.reshape((60000, 784))
a = a.reshape((10000, 784))

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

sc = StandardScaler()

x_std = sc.fit_transform(x)


In [4]:
x_t, x_test, y_t, y_test =train_test_split(x_std, y, test_size=0.1)
x_train, x_valid, y_train, y_valid = train_test_split(x_t, y_t, test_size=0.1)
print (x_train.shape, y_train.shape)
print (x_valid.shape, y_valid.shape)
print (x_test.shape, y_test.shape)

(48600, 784) (48600,)
(5400, 784) (5400,)
(6000, 784) (6000,)


In [20]:
from X_py_boost import GradientBoosting

model2 = GradientBoosting(loss='CE_2', metric='accuracy', ntrees=10000, es=20, max_depth=8)
model2.fit(x_train, y_train, eval_sets=[{'X': x_valid, 'y': y_valid},])


model3 = GradientBoosting(loss='CE_3', metric='accuracy', ntrees=10000, es=20, max_depth=8)
model3.fit(x_train, y_train, eval_sets=[{'X': x_valid, 'y': y_valid},])



[12:50:51] Stdout logging level is INFO.
[12:50:51] GDBT train starts. Max iter 10000, early stopping rounds 20
[12:50:53] Iter 0; Sample 0, Accuracy = 0.8144444227218628; 
[12:50:57] Iter 10; Sample 0, Accuracy = 0.9222221970558167; 
[12:51:00] Iter 20; Sample 0, Accuracy = 0.9383333325386047; 
[12:51:04] Iter 30; Sample 0, Accuracy = 0.9468518495559692; 
[12:51:08] Iter 40; Sample 0, Accuracy = 0.9527778029441833; 
[12:51:12] Iter 50; Sample 0, Accuracy = 0.9585185050964355; 
[12:51:15] Iter 60; Sample 0, Accuracy = 0.9629629850387573; 
[12:51:19] Iter 70; Sample 0, Accuracy = 0.9653703570365906; 
[12:51:23] Iter 80; Sample 0, Accuracy = 0.9677777886390686; 
[12:51:27] Iter 90; Sample 0, Accuracy = 0.9698148369789124; 
[12:51:30] Iter 100; Sample 0, Accuracy = 0.9701851606369019; 
[12:51:34] Iter 110; Sample 0, Accuracy = 0.9722222089767456; 
[12:51:38] Iter 120; Sample 0, Accuracy = 0.9724074006080627; 
[12:51:41] Iter 130; Sample 0, Accuracy = 0.9731481671333313; 
[12:51:45] Iter 1

In [21]:
import plotly.graph_objects as go

y_G2 = get_results(model2)
y_G3 = get_results(model3)
xs = [i for i in range(max(len(y_G2), len(y_G3)))]

fig = go.Figure(layout_title_text="2nd and 3rd order training, MNIST, accuracy")
fig.add_trace(go.Scatter(x=xs, y=y_G2,
                    mode='lines',
                    name='2nd order'))
fig.add_trace(go.Scatter(x=xs, y=y_G3,
                    mode='lines',
                    name='3rd order'))

fig.show()